In [11]:
!pip -q install smolagents

In [10]:
!pip -q install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 6.5 MB/s eta 0:00:00a 0:00:01


In [12]:
from openai import OpenAI
from smolagents import Tool, HfApiModel, ToolCallingAgent
from pinecone import Pinecone

In [ ]:
client = OpenAI(api_key="fff", base_url="https://api.deepseek.com")

In [14]:
systemPrompt = """You are ZenCode, an expert AI assistant specializing in generating React applications that strictly adhere to enterprise design standards and component libraries.

<repository_context>
  You will receive a serialized context of the current repository's generated code state. This includes:
  - File structure and contents of previously generated code
  - Current state of all files and directories
  - Previous modifications and their history
  - Internal Library, an existing react project contained in your knowledge base that you should analyse and reuse code from, which you can access with the 'retriever' tool,
.
  
  CRITICAL:
  - All file operations must be performed relative to this existing context
  - Maintain consistency with previously generated code
  - Consider dependencies and relationships between existing files
  - Ensure backwards compatibility when modifying existing files
  - make sure to ckeck the Internal Library using the retriever tool
</repository_context>

<system_constraints>
  You are operating in WebContainer, an in-browser Node.js runtime that emulates a Linux system. Key limitations:
  - Runs in browser, not a full Linux system
  - Can only execute browser-compatible code (JS, WebAssembly)
  - Python limited to standard library only (NO pip)
  - No native binary execution or C/C++ compilation
  - Git not available
  - For databases, use browser-compatible options (libsql, sqlite)
</system_constraints>

<enterprise_context>
  You will receive:
  1. Top-K relevant internal components from Internal library using the retriever tool
  2. Global CSS standards and design system guidelines
  3. Approved list of npm packages
  
  CRITICAL REQUIREMENTS:
  - ONLY use provided internal components from the Internal library
  - Strictly follow enterprise design standards
  - Only use approved npm packages. Do not use any external npm packages not used in the given context. 
  - Focus on rapid prototyping by reusing existing components
  - Maintain consistent styling and UX patterns
</enterprise_context>

<response_format>
  Generate responses in JSON format following the StepType enum:
  - CreateFile (0): New files
  - CreateFolder (1): New directories
  - EditFile (2): Modify files
  - DeleteFile (3): Remove files

  Each step must include:
  - id: Unique integer
  - title: Step description
  - description: Detailed explanation
  - type: StepType enum value
  - content: File content or command
  - path: Target file/folder path
</response_format>

<code_formatting>
  - Use 2 spaces for indentation
  - Follow enterprise code style guide
  - Split functionality into modular components
  - Keep files small and focused
  - Use proper TypeScript types
</code_formatting>

IMPORTANT:
1. Think holistically before generating responses
2. Consider all file dependencies and impacts
3. Always use latest file modifications
4. Install dependencies first
5. Never re-run dev server if already running
6. Provide complete, untruncated code
7. Focus on reusing enterprise components
   
"""

In [15]:
codeBasePrompt = """
Project Files:\n\nThe following is a list of all project files and their complete contents that are currently visible and accessible to you.\n\neslint.config.js:\n```\nimport js from '@eslint/js';\nimport globals from 'globals';\nimport reactHooks from 'eslint-plugin-react-hooks';\nimport reactRefresh from 'eslint-plugin-react-refresh';\nimport tseslint from 'typescript-eslint';\n\nexport default tseslint.config(\n  { ignores: ['dist'] },\n  {\n    extends: [js.configs.recommended, ...tseslint.configs.recommended],\n    files: ['**/*.{ts,tsx}'],\n    languageOptions: {\n      ecmaVersion: 2020,\n      globals: globals.browser,\n    },\n    plugins: {\n      'react-hooks': reactHooks,\n      'react-refresh': reactRefresh,\n    },\n    rules: {\n      ...reactHooks.configs.recommended.rules,\n      'react-refresh/only-export-components': [\n        'warn',\n        { allowConstantExport: true },\n      ],\n    },\n  }\n);\n\n```\n\nindex.html:\n```\n<!doctype html>\n<html lang=\"en\">\n  <head>\n    <meta charset=\"UTF-8\" />\n    <link rel=\"icon\" type=\"image/svg+xml\" href=\"/vite.svg\" />\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n    <title>Vite + React + TS</title>\n  </head>\n  <body>\n    <div id=\"root\"></div>\n    <script type=\"module\" src=\"/src/main.tsx\"></script>\n  </body>\n</html>\n\n```\n\npackage.json:\n```\n{\n  \"name\": \"vite-react-typescript-starter\",\n  \"private\": true,\n  \"version\": \"0.0.0\",\n  \"type\": \"module\",\n  \"scripts\": {\n    \"dev\": \"vite\",\n    \"build\": \"vite build\",\n    \"lint\": \"eslint .\",\n    \"preview\": \"vite preview\"\n  },\n  \"dependencies\": {\n    \"lucide-react\": \"^0.344.0\",\n    \"react\": \"^18.3.1\",\n    \"react-dom\": \"^18.3.1\"\n  },\n  \"devDependencies\": {\n    \"@eslint/js\": \"^9.9.1\",\n    \"@types/react\": \"^18.3.5\",\n    \"@types/react-dom\": \"^18.3.0\",\n    \"@vitejs/plugin-react\": \"^4.3.1\",\n    \"autoprefixer\": \"^10.4.18\",\n    \"eslint\": \"^9.9.1\",\n    \"eslint-plugin-react-hooks\": \"^5.1.0-rc.0\",\n    \"eslint-plugin-react-refresh\": \"^0.4.11\",\n    \"globals\": \"^15.9.0\",\n    \"postcss\": \"^8.4.35\",\n    \"tailwindcss\": \"^3.4.1\",\n    \"typescript\": \"^5.5.3\",\n    \"typescript-eslint\": \"^8.3.0\",\n    \"vite\": \"^5.4.2\"\n  }\n}\n\n```\n\npostcss.config.js:\n```\nexport default {\n  plugins: {\n    tailwindcss: {},\n    autoprefixer: {},\n  },\n};\n\n```\n\nsrc/App.tsx:\n```\nimport React from 'react';\n\nfunction App() {\n  return (\n    <div className=\"min-h-screen bg-gray-100 flex items-center justify-center\">\n      <p>Start prompting (or editing) to see magic happen :)</p>\n    </div>\n  );\n}\n\nexport default App;\n\n```\n\nsrc/index.css:\n```\n@tailwind base;\n@tailwind components;\n@tailwind utilities;\n\n```\n\nsrc/main.tsx:\n```\nimport { StrictMode } from 'react';\nimport { createRoot } from 'react-dom/client';\nimport App from './App.tsx';\nimport './index.css';\n\ncreateRoot(document.getElementById('root')!).render(\n  <StrictMode>\n    <App />\n  </StrictMode>\n);\n\n```\n\nsrc/vite-env.d.ts:\n```\n/// <reference types=\"vite/client\" />\n\n```\n\ntailwind.config.js:\n```\n/** @type {import('tailwindcss').Config} */\nexport default {\n  content: ['./index.html', './src/**/*.{js,ts,jsx,tsx}'],\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n};\n\n```\n\ntsconfig.app.json:\n```\n{\n  \"compilerOptions\": {\n    \"target\": \"ES2020\",\n    \"useDefineForClassFields\": true,\n    \"lib\": [\"ES2020\", \"DOM\", \"DOM.Iterable\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n    \"jsx\": \"react-jsx\",\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"src\"]\n}\n\n```\n\ntsconfig.json:\n```\n{\n  \"files\": [],\n  \"references\": [\n    { \"path\": \"./tsconfig.app.json\" },\n    { \"path\": \"./tsconfig.node.json\" }\n  ]\n}\n\n```\n\ntsconfig.node.json:\n```\n{\n  \"compilerOptions\": {\n    \"target\": \"ES2022\",\n    \"lib\": [\"ES2023\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"vite.config.ts\"]\n}\n\n```\n\nvite.config.ts:\n```\nimport { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\n// https://vitejs.dev/config/\nexport default defineConfig({\n  plugins: [react()],\n  optimizeDeps: {\n    exclude: ['lucide-react'],\n  },\n});\n\n```\n\nHere is a list of files that exist on the file system but are not being shown to you:\n\n  - .gitignore\n  - package-lock.json\n  - .bolt/prompt"""

In [ ]:
pc = Pinecone("fff")
index = pc.Index("llama-text-embed-v2-index")

In [ ]:
# class RetrieverTool(Tool):
#     name = "retriever"
#     description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
#     inputs = {
#         "query": {
#             "type": "string",
#             "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
#         }
#     }
#     output_type = "string"

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#     def forward(self, query: str) -> str:
#         assert isinstance(query, str), "Your search query must be a string"
#         queryEmbedding = pc.inference.embed(
#             model="llama-text-embed-v2",
#             inputs=[query],
#             parameters={
#                 "input_type": "passage"
#             }
#         )
        
#         docs = index.query(            
#             namespace="ns1",
#             vector=queryEmbedding[0]['values'],
#             top_k=3,
#             include_values=True,
#             include_metadata=True,
# )

#         return "\nRetrieved documents:\n" + "".join(
#             [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
#         )

In [29]:
def retriever(query: str) -> str:
    assert isinstance(query, str), "Your search query must be a string"
    queryEmbedding = pc.inference.embed(
        model="llama-text-embed-v2",
        inputs=[query],
        parameters={
            "input_type": "passage"
        }
    )
    
    docs = index.query(            
        namespace="ns1",
        vector=queryEmbedding[0]['values'],
        top_k=3,
        include_values=True,
        include_metadata=True,
    )
    # return "\nRetrieved documents:\n" + "".join(
    # [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)])
    return docs

In [22]:
query = "add a navbar to the react project"

In [26]:
from openai import OpenAI

def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message


tools = [
    {
        "type": "function",
        "function": {
            "name": "retriever",
            "description": "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
                    }
                },
                "required": ["query"]
            },
        }
    },
]

messages=[
    {"role": "system", "content": systemPrompt},
    {"role": "user", "content": codeBasePrompt},
    {"role": "user", "content": query}
]
message = send_messages(messages)
tool = message.tool_calls
print(f"initial Model>\t {message.content}")
messages.append(message)
print(tool)


initial Model>	 
[ChatCompletionMessageToolCall(id='call_0_866f91c9-4059-4c0a-8a7a-02f25864f98e', function=Function(arguments='{"query":"enterprise navbar component"}', name='retriever'), type='function', index=0)]


In [27]:
message

ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_866f91c9-4059-4c0a-8a7a-02f25864f98e', function=Function(arguments='{"query":"enterprise navbar component"}', name='retriever'), type='function', index=0)])

In [30]:
retriever("enterprise navbar component")

{'matches': [{'id': '84d3ac48-0881-11f0-8b6a-0242ac130202',
              'metadata': {'component': '{"name": "Title", "description": "A '
                                        'React functional component that '
                                        'renders a styled heading with the '
                                        'provided text.", "returnType": '
                                        '"JSX.Element", "inputParams": '
                                        '[{"type": "TitleProps", "name": '
                                        '"text", "description": "The text to '
                                        'be displayed inside the heading."}], '
                                        '"import": "import { Title } from '
                                        '\'src/components/ui/title/title\';", '
                                        '"example": "import { Title } from '
                                        "'src/components/ui/title/title';\\n\\nconst "
        

In [ ]:
# messages.append({"role": "tool", "tool_call_id": tool.id, "content": "24℃"})
# message = send_messages(messages)
# print(f"Model>\t {message.content}")